In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline, AutoConfig
import shap

INPUT_PATH = '../../models/title'
INPUT_TEXT = "Meter el título y -- el resumen" # -- for adding the summary
if __name__ == '__main__':
    model = AutoModelForSequenceClassification.from_pretrained(INPUT_PATH)
    tokenizer = AutoTokenizer.from_pretrained(INPUT_PATH)
    config = AutoConfig.from_pretrained(INPUT_PATH)
    pipeline = pipeline('text-classification', model=model, tokenizer=tokenizer, config=config, return_all_scores=True)
    explainer = shap.Explainer(pipeline)
    shap_values = explainer([INPUT_TEXT])
    shap.plots.text(shap_values[0, :, 'LABEL_0'])


  0%|          | 0/48 [00:00<?, ?it/s]